# Simple Image Classifier

Dis notebook go show you how you fit take classify images wit pre-trained neural network.

**Wetin you go learn:**
- How to load and use pre-trained model
- How to preprocess image
- How to make predictions for images
- How to understand confidence scores

**Use case:** To sabi wetin dey inside images (like "cat", "dog", "car", etc.)

---


## Step 1: Import Di Libraries Wey We Need

Make we bring di tools wey we go use. No worry if you no sabi all of dem yet!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Step 2: Load Pre-trained Model

We go use **MobileNetV2**, one kain neural network wey dem don train already wit millions of images.

Dis one na wetin dem dey call **Transfer Learning** - to use model wey another person don train!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Step 3: Helper Functions

Make we create functions wey go help us load and prepare images for our model.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Step 4: Test am for Sample Images

Make we try classify some images wey dey internet!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Classify Each Image


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Step 5: Try Your Own Images!

Change di URL wey dey below wit any image URL wey you wan use classify.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Wetin Just Happen?

1. **We load one model wey dem don train before** - MobileNetV2 na model wey dem don train with millions of pictures.
2. **We preprocess di pictures** - We resize and arrange dem well for di model.
3. **Di model predict somtin** - E give probabilities for 1000 different object classes.
4. **We decode di results** - We change di numbers to labels wey humans fit understand.

### How Confidence Scores Work

- **90-100%**: Di model sure wella (e almost dey correct).
- **70-90%**: Di model dey sure (e fit dey correct).
- **50-70%**: Di model dey somehow sure (e fit dey correct).
- **Below 50%**: Di model no too sure (e dey uncertain).

### Why predictions fit no correct?

- **Wey di angle or light no dey normal** - Di model na for normal pictures dem train am.
- **Plenty objects dey di picture** - Di model dey expect say na one main object go dey.
- **Rare objects dey di picture** - Di model sabi only 1000 categories.
- **Di picture no clear** - If e dey blurry or e no get better quality, e go hard for di model. 

---


## 🚀 Next Steps

1. **Try diffren kain foto:**
   - Go find foto for [Unsplash](https://unsplash.com)
   - Right-click → "Copy image address" to get di URL

2. **Try diffren tins:**
   - Wetin go happen if na abstract art?
   - E fit sabi objects wey dey diffren angles?
   - How e go take handle plenty objects?

3. **Learn more:**
   - Check [Computer Vision lessons](../lessons/4-ComputerVision/README.md)
   - Learn how to train your own image classifier
   - Understand how CNNs (Convolutional Neural Networks) dey work

---

## 🎉 Congratulations!

You don just build image classifier wey use beta neural network!

Dis same technique dey power:
- Google Photos (wey dey arrange your photos)
- Self-driving cars (wey dey sabi objects)
- Medical diagnosis (wey dey check X-rays)
- Quality control (wey dey find defects)

Make you continue to dey explore and learn! 🚀


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don translate wit AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). Even as we dey try make am accurate, abeg sabi say automatic translation fit get mistake or no dey correct well. Di original dokyument for im native language na di main source wey you go fit trust. For important information, e good make professional human translation dey use. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
